In [30]:
import pyrvo2
import numpy as np
import math

In [31]:
from pyrvo2 import *

In [48]:
goals=[];
def  setupScenario(sim):
    sim.setTimeStep(0.25);
    #sim.setAgentDefaults(15.0, 10, 10.0, 10.0, 1.5, 2.0,np.array([0,0]));
    sim.setAgentDefaults(15.0, 10, 10.0, 10.0, 3.0, 2.0,np.array([0,0]));
    
    #for i in range(250):
    #    sim.addAgent(200.0 *np.array([math.cos(i * 2.0 * math.pi / 250.0),
    #                               math.sin(i * 2.0 *  math.pi / 250.0)]))
    #    goals.append(-sim.getAgentPosition(i));
    
    n_grp_y = 5;               # number of agent of each group
    n_grp_yh = (n_grp_y-1)/2;
    n_grp_x = 5;               # number of agent of each group
    n_grp_xh = (n_grp_x-1)/2;
    dist_grp = 9;           # distance between next agent
    
    for i in range(n_grp_y):
        yg = (i-n_grp_yh) * dist_grp
        for j in range(n_grp_x):
            xg_offset = (j-n_grp_xh) * dist_grp
            sim.addAgent(np.array([-150-xg_offset,yg]))
            sim.addAgent(np.array([ 150-xg_offset,yg]))    
            goals.append(np.array([ 150-xg_offset,yg]))
            goals.append(np.array([-150-xg_offset,yg]))

        
def updateVisualization(sim):
    print(sim.getGlobalTime());
    for i in range(sim.getNumAgents()):
        print(sim.getAgentPosition())

def setPreferredVelocities( sim):
    for i in range(sim.getNumAgents()):
        goalVector = goals[i] - sim.getAgentPosition(i);
        if (np.linalg.norm(goalVector) > 1.0):
            goalVector = goalVector/np.linalg.norm(goalVector)
        sim.setAgentPrefVelocity(i, goalVector);

def reachedGoal(sim):
    for i in range(sim.getNumAgents()):
        if (np.linalg.norm(sim.getAgentPosition(i) - goals[i]) > sim.getAgentRadius(i)):
            return False;
    return True;

In [49]:
import pygame as pg
screen_size=[400,400]
world_size =[500,500]
screen=pg.display.set_mode(screen_size)
screen_size= np.array(screen_size)
screen_center = screen_size/2
world_center = np.array([0,0])
world_size = np.array(world_size)
def getScreenPos(world_pos):
    return (world_pos-world_center)/world_size*screen_size+screen_center
def getScreenRadius(radius_world):
    return radius_world/np.max(world_size)*np.max(screen_size)

def visualize(sim):
    screen.fill(pg.Color("white"))
    for i in range(sim.getNumAgents()):
        pos = getScreenPos(sim.getAgentPosition(i))
        radius = getScreenRadius(sim.getAgentRadius(i))
        pg.draw.circle(screen,pg.Color("blue"),(pos[0],pos[1]),radius,0)
    pg.display.update()

In [50]:
import time
start_time = time.time()
sim = RVOSimulator();
setupScenario(sim);
while (not reachedGoal(sim)):
    setPreferredVelocities(sim);
    sim.step();
    visualize(sim)
end_time = time.time()
print(end_time-start_time)

15.32560920715332
